In [84]:
import ccxt, abc
from statistics import mean

In [62]:
class Classifier(abc.ABC):
    def __init__(self):
        self.exchange = ccxt.coinsecure()
        super().__init__()
    
    @abc.abstractmethod
    def predict(self): pass

In [134]:
class _Window(list):
    def __init__(self, window_size=20):
        self.__window_size, self.__actual_window_size = window_size, 0
        super().__init__()
        
    def append(self, price):
        super().append(price)
        if self.__window_size > self.__actual_window_size: self.__actual_window_size += 1
        else: del self[0]
            
class SMA(Classifier):
    def __init__(self, window_size=20):
        global __Window
        self.__window_size = window_size
        self.__sp_window, self.__bp_window = _Window(window_size), _Window(window_size)
        super().__init__()
    
    def predict(self):
        def classify(avg, price):
            if price < avg: return -1
            if price > avg: return 1
            return 0
        #response = self.exchange.fetchTicker('BTC/INR')
        #to do -- convert response to a tuple of SP and BP
        sp, bp = 0, 0
        self.__sp_window.append(sp), self.__bp_window.append(bp)
        return classify(mean(self.__sp_window), sp), classify(mean(self.__bp_window), bp)

In [150]:
predictor = SMA(3)

In [161]:
print(predictor._SMA__sp_window)

[0, 0, 0]


In [160]:
predictor.predict()

(0, 0)